# Task 2: 
You have to push the original data in a structured format into MongoDB. The following structure is to be followed.

In [158]:
#Libraries
import pandas as pd
import numpy as np
# !pip install pymongo
from pymongo import MongoClient

In [159]:
#Initiate client
client = MongoClient("mongodb://localhost:27017")

In [160]:
#Import dataset
df = pd.read_csv("insurance_data.csv", delimiter=';')

#Convert to correct value for insertion
def change_value(val, to_change1, new_val1, to_change2, new_val2):
        val = str(val)
        if type(to_change1) != list: 
            to_change1 = str(to_change1)
        if type(to_change2) != list:
            to_change2 = str(to_change2)

        if val in list(to_change1) or val == to_change1:
            return new_val1
        elif val in list(to_change2) or val == to_change2:
            return new_val2
        else:
            return val

#Change to correct data value for insertion
df['is45OrOlder'] = df.is45OrOlder.apply(lambda x: change_value(x, 1.0, True, 0.0, False))
df['isMarried'] = df.isMarried.apply(lambda x: change_value(x, 'Yes', True, 'No', False))
df['hasKids'] = df.hasKids.apply(lambda x: change_value(x, 'Yes', True, 'No', False))
df['termLifeInsurance'] = df.termLifeInsurance.apply(lambda x: change_value(x, 'Yes', True, ['No','no term life policy taken','term life policy not taken'], False))
df['multipleTermLifePolicies'] = df.multipleTermLifePolicies.apply(lambda x: change_value(x, 'Yes', True, 'No', False))
df['healthInsurance'] = df.healthInsurance.apply(lambda x: change_value(x, ['Yes','Class A','Class B'], True, 'No', False))
df['eStatements'] = df.eStatements.apply(lambda x: change_value(x, 'Yes', True, 'No', False))
df['renewal'] = df.renewal.apply(lambda x: change_value(x, 'Y', True, 'N', False))
df['monthlyPremium'] = df['monthlyPremium'].str.replace(',', '.')
df['totalPremium'] = df['totalPremium'].str.replace(',', '.')
df['totalPremium'] = df['totalPremium'].replace(' ', np.nan)

In [161]:
#Convert df to dictionary for insertion into MongoDB
data = df.to_dict("records")

#Create database
db = client['VICA']

#Insert df into database as collection
db.Insurance.insert_many(data)

In [162]:
#Convert to correct data type
#str to float
db.Insurance.update_many(
    {'monthlyPremium': {'$type': 2}},
    [{'$set':{'monthlyPremium':{'$toDouble':'$monthlyPremium'}}}]
)

#str to float
db.Insurance.update_many(
    {'totalPremium': {'$type': 2}},
    [{'$set':{'totalPremium':{'$toDouble':'$totalPremium'}}}]
)

#str to array
db.Insurance.update_many(
  {'healthRiders': {'$type': 2}},
  [{ '$set': { 'healthRiders': { "$split": [ "$healthRiders", "," ] } } }],
)